In [1]:
import datetime
import os 
from dotenv import load_dotenv
import numpy as np
import pandas as pd

path = '/home/rafaelfabrichimidt/Documentos/projetos/env_vars/'
envfile = '.env'

load_dotenv(dotenv_path = path + envfile)

True

In [5]:
import duckdb
conn = duckdb.execute("ATTACH 'dbname=postgresql_oltp user=admin host=localhost, password=12345' AS db (TYPE POSTGRES); ")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [42]:
df_insc = conn.sql('''
                 SELECT nu_inscricao, 
                        nu_nota_comp1, 
                        nu_nota_comp2, 
                        nu_nota_comp3, 
                        nu_nota_comp4,
                        nu_nota_comp5,
                        nu_nota_redacao 
                 FROM db.enem.base_enem
                 WHERE nu_nota_redacao IS NOT NULL
                 LIMIT 100''')

#so salva na primeira vez
#df_insc.df().to_csv('Experimento_1.csv', index = False)

In [6]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

from openai import OpenAI
client = OpenAI()

In [7]:
def message_LLM_Sql(user_message_question):
    
    system_prompt_sql = 'Você é responsável por construir queries SQL Postgres, para a query SQL funcionar deve-se adicionar no FROM database.schema.table. Retorne apenas a query SQL'

    descricao_base_enem_sql = '''
                                    database=db;
                                    schema=enem;
                                    table=base_enem;
                                    campo no_municipio_esc tipo varchar Nome do município da escola;
                                    campo sg_uf_esc tipo varchar Sigla da Unidade da Federação da escola;
                                    campo nu_inscricao tipo int Número da inscrição do aluno; 
                                    campo tp_status_redacao tipo int Situação da redação do participante;
                                    campo nu_nota_comp1 tipo int Nota da competência 1 da prova de redação; 
                                    campo nu_nota_comp2 tipo int Nota da competência 2 da prova de redação; 
                                    campo nu_nota_comp3 tipo int Nota da competência 3 da prova de redação;
                                    campo nu_nota_comp4 tipo int Nota da competência 4 da prova de redação; 
                                    campo nu_nota_comp5 tipo int Nota da competência 5 da prova de redação; 
                                    campo nu_nota_redacao tipo int Nota final da prova de redação ;
                                    campo com nome Q006 tipo varchar Traz informação de qual é a renda mensal da família do aluno;

    '''


    message = [
        {'role': 'system', 'content': system_prompt_sql},
        {'role': 'system', 'content': descricao_base_enem_sql},
        {'role': 'user', 'content': user_message_question}
    ]

    return message



In [8]:
def message_LLM_Teacher(user_mensage_with_data):
    system_prompt_dialogue = '''
Você é um professor responsável por analisar o desempenho de um aluno na redação do Enem e recomendar estudos de acordo com a pontuação deste aluno com foco na dúvida que ele apresentou.
Além disso, resuma as Competência de cada área, não precisa repetir as competências ou trazer um exemplo completo de redação nota 1000 apenas descrever e sugerir exemplos de melhoria em cada competência.
Importante ser sugestões bem fundamentadas!
    '''


    descricao_detalhada_enem = open(file = 'descricao_detalhada.txt', mode = 'r').read()

    message = [
        {'role': 'system', 'content': system_prompt_dialogue},
        {'role': 'system', 'content': descricao_detalhada_enem},                                
        {'role': 'user', 'content': user_mensage_with_data}
    ]

    return message

In [25]:
def save_results_from_llm_sql(dic, experimento, name):
    df = pd.DataFrame.from_dict(data = dic, orient = 'index').reset_index()
    df.columns = ['numero_inscricao', 'query']
    df.to_csv(f'Resultados_Experimentos/{experimento}/{name}', index = False)

def save_results_from_llm_teacher(dic):
    df = pd.DataFrame.from_dict(data = dic, orient = 'index').reset_index()
    df.columns = ['numero_inscricao', 'recomendacao_llm']
    return df

def request(df, experimento, name):
    lista = []
    for sql_query in df['query'].values:
        try:
            df_query = conn.sql(sql_query)
            #print(df_query.fetchnumpy())
            lista.append('Sucesso')
        except:
            print('erro')
            lista.append('Falha')

    df['requisicao'] = lista

    df.to_csv(f'Resultados_Experimentos/{experimento}/{name}', index = False)

## Experimento 1 

In [129]:
alunos_exp_1 = pd.read_csv('Experimento_1.csv')
nu_inscricao_aluno_exp_1 = alunos_exp_1['nu_inscricao'].values

nu_inscricao_aluno_exp_1.shape

(100,)

In [131]:
user_message_question = 'Sou o aluno com inscrição {} desejava saber meu desempenho na Redação do Enem e como posso melhorar no próximo ano?'

In [132]:
dic = {}

i = 0

for numero_inscricao in nu_inscricao_aluno_exp_1:

    message = user_message_question.format(numero_inscricao)

    print(message)

    resposta = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=message_LLM_Sql(message),
        max_tokens=200,
        temperature=0.7,
    )

    mensagem_resp = resposta.choices[0].message.content
    
    if i < 5: 
        print(mensagem_resp)

    sql_query = mensagem_resp.replace("```", "").split('sql')[1]

    dic[str(numero_inscricao)] = sql_query

    i = i + 1

Sou o aluno com inscrição 210061103945 desejava saber meu desempenho na Redação do Enem e como posso melhorar no próximo ano?
```sql
SELECT 
    no_municipio_esc, 
    sg_uf_esc, 
    nu_inscricao, 
    tp_status_redacao, 
    nu_nota_comp1, 
    nu_nota_comp2, 
    nu_nota_comp3, 
    nu_nota_comp4, 
    nu_nota_comp5, 
    nu_nota_redacao 
FROM 
    db.enem.base_enem 
WHERE 
    nu_inscricao = 210061103945;
```
Sou o aluno com inscrição 210060214087 desejava saber meu desempenho na Redação do Enem e como posso melhorar no próximo ano?
```sql
SELECT nu_inscricao, tp_status_redacao, nu_nota_comp1, nu_nota_comp2, nu_nota_comp3, nu_nota_comp4, nu_nota_comp5, nu_nota_redacao
FROM db.enem.base_enem
WHERE nu_inscricao = 210060214087;
```
Sou o aluno com inscrição 210059980948 desejava saber meu desempenho na Redação do Enem e como posso melhorar no próximo ano?
```sql
SELECT nu_inscricao, tp_status_redacao, nu_nota_comp1, nu_nota_comp2, nu_nota_comp3, nu_nota_comp4, nu_nota_comp5, nu_nota_r

In [134]:
#Salvar resultados
save_results_from_llm_sql(dic , 'Experimento_1', 'query_sql.csv')

In [135]:
#Ler resultados
resultado_experimento_1 = pd.read_csv('Resultados_Experimentos/Experimento_1/query_sql.csv')
resultado_experimento_1

,numero_inscricao,query
0,210061103945,"\nSELECT \n no_municipio_esc, \n sg_uf_e..."
1,210060214087,"\nSELECT nu_inscricao, tp_status_redacao, nu_n..."
2,210059980948,"\nSELECT nu_inscricao, tp_status_redacao, nu_n..."
3,210060801601,"\nSELECT \n no_municipio_esc, \n sg_uf_e..."
4,210059085130,"\nSELECT \n no_municipio_esc,\n sg_uf_es..."
...,...,...
95,210059527738,"\nSELECT \n nu_inscricao, \n nu_nota_com..."
96,210059388095,"\nSELECT \n no_municipio_esc, \n sg_uf_e..."
97,210059980951,"\nSELECT \n no_municipio_esc, \n sg_uf_e..."
98,210058061542,"\nSELECT no_municipio_esc, sg_uf_esc, tp_statu..."


In [ ]:
request(resultado_experimento_1, 'Experimento_1', 'query_sql_request.csv')

In [145]:
df = pd.read_csv('Resultados_Experimentos/Experimento_1/query_sql_request_consolidado.csv')
df


,numero_inscricao,query,requisicao
0,210061103945,"\nSELECT \n no_municipio_esc, \n sg_uf_e...",Sucesso
1,210060214087,"\nSELECT nu_inscricao, tp_status_redacao, nu_n...",Sucesso
2,210059980948,"\nSELECT nu_inscricao, tp_status_redacao, nu_n...",Sucesso
3,210060801601,"\nSELECT \n no_municipio_esc, \n sg_uf_e...",Sucesso
4,210059085130,"\nSELECT \n no_municipio_esc,\n sg_uf_es...",Sucesso
...,...,...,...
95,210059527738,"\nSELECT \n nu_inscricao, \n nu_nota_com...",Sucesso
96,210059388095,"\nSELECT \n no_municipio_esc, \n sg_uf_e...",Sucesso
97,210059980951,"\nSELECT \n no_municipio_esc, \n sg_uf_e...",Sucesso
98,210058061542,"\nSELECT no_municipio_esc, sg_uf_esc, tp_statu...",Sucesso


In [ ]:
dic = {}

tam = int( (df.shape[0] * 0.2) + 1 )

for i in range(0, tam, 1):
    print(i)
    if df.loc[i, 'requisicao'] == 'Sucesso':

        numero_inscricao = df.loc[i, 'numero_inscricao']
        df_query = conn.sql(df.loc[i, 'query'])

        user_message_with_data = user_message_question.format(numero_inscricao) + 'Os dados com detalhes do desempenho do Aluno encontra-se abaixo \n ' + f' {df_query.fetchnumpy()} Ajude o aluno para melhorar sua nota: '

        resposta = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=message_LLM_Teacher(user_message_with_data),
            max_tokens=700,
            temperature=0.7,
        )

        mensagem_resp = resposta.choices[0].message.content

        #print(mensagem_resp)

        dic[str(numero_inscricao)] = mensagem_resp


In [151]:
dic

{'210061103945': 'Caro aluno,\n\nParabéns pelo seu desempenho na redação do Enem, onde você obteve uma nota total de 700 pontos. Vamos analisar suas notas em cada competência e oferecer sugestões específicas para que você possa melhorar no próximo ano.\n\n### Análise do Desempenho\n\n1. **Competência 1: Domínio da modalidade escrita formal da Língua Portuguesa**\n   - **Nota:** 140 pontos (Bom domínio)\n   - **Análise:** Você demonstra um bom domínio da escrita formal, mas ainda apresenta alguns desvios gramaticais. \n   - **Sugestões de Melhoria:**\n     - Pratique a leitura e a escrita de diferentes gêneros textuais. \n     - Revise e estude as regras gramaticais e de pontuação, focando em aspectos que você considera mais desafiadores.\n     - Considere participar de grupos de escrita ou workshops que possam ajudar a aprimorar seu estilo.\n\n2. **Competência 2: Compreensão e desenvolvimento do tema**\n   - **Nota:** 200 pontos (Excelente domínio)\n   - **Análise:** Você teve um desem

In [152]:
df_llm_teacher = save_results_from_llm_teacher(dic)
df_llm_teacher['numero_inscricao'] = df_llm_teacher['numero_inscricao'].astype('int')
resultado = pd.merge(left = df, right = df_llm_teacher, how = 'left', on = 'numero_inscricao')
resultado.to_csv('Resultados_Experimentos/Experimento_1/resultado_final_recomendacao.csv', index = False)

## Experimento 2

In [153]:
df_insc = conn.sql('''
                 SELECT nu_inscricao, 
                        nu_nota_comp1, 
                        nu_nota_comp2, 
                        nu_nota_comp3, 
                        nu_nota_comp4,
                        nu_nota_comp5,
                        nu_nota_redacao 
                 FROM db.enem.base_enem
                 WHERE nu_nota_redacao IS NOT NULL
                 LIMIT 500''')

#so salva na primeira vez
#df_insc.df().to_csv('Experimento_2.csv', index = False)

In [193]:
alunos_exp_2 = pd.read_csv('Experimento_2.csv')
nu_inscricao_aluno_exp_2 = alunos_exp_2['nu_inscricao'].values
#nu_inscricao_aluno_exp_2 = nu_inscricao_aluno_exp_2[0:5]
nu_inscricao_aluno_exp_2.shape

(500,)

In [194]:
user_message_question = '''
Sou o aluno com inscrição {} desejava saber meu desempenho na Redação do Enem?
Também queria saber como esta meu desempenho comparado ao desempenho médio no Estado que moro e a Cidade?'''

In [195]:
dic = {}

i = 0

for numero_inscricao in nu_inscricao_aluno_exp_2:

    message = user_message_question.format(numero_inscricao)

    print(message)

    resposta = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=message_LLM_Sql(message),
        max_tokens=1000,
        temperature=0.7,
    )

    mensagem_resp = resposta.choices[0].message.content
    
    if i < 5: 
        print(mensagem_resp)

    sql_query = mensagem_resp.replace("```", "").split('sql')[1]

    dic[str(numero_inscricao)] = sql_query

    i = i + 1


Sou o aluno com inscrição 210061103945 desejava saber meu desempenho na Redação do Enem?
Também queria saber como esta meu desempenho comparado ao desempenho médio no Estado que moro e a Cidade?
```sql
SELECT 
    e.nu_inscricao,
    e.nu_nota_redacao,
    e.no_municipio_esc,
    e.sg_uf_esc,
    AVG(e2.nu_nota_redacao) AS media_estado,
    AVG(e3.nu_nota_redacao) AS media_cidade
FROM 
    db.enem.base_enem e
LEFT JOIN 
    db.enem.base_enem e2 ON e.sg_uf_esc = e2.sg_uf_esc
LEFT JOIN 
    db.enem.base_enem e3 ON e.no_municipio_esc = e3.no_municipio_esc
WHERE 
    e.nu_inscricao = 210061103945
GROUP BY 
    e.nu_inscricao, e.no_municipio_esc, e.sg_uf_esc;
```

Sou o aluno com inscrição 210060214087 desejava saber meu desempenho na Redação do Enem?
Também queria saber como esta meu desempenho comparado ao desempenho médio no Estado que moro e a Cidade?
```sql
SELECT 
    b.no_municipio_esc,
    b.sg_uf_esc,
    b.nu_inscricao,
    b.nu_nota_redacao,
    AVG(b.nu_nota_redacao) OVER (PART

In [196]:
#Salvar resultados
save_results_from_llm_sql(dic , 'Experimento_2', 'query_sql.csv')

#Ler resultados
resultado_experimento_2 = pd.read_csv('Resultados_Experimentos/Experimento_2/query_sql.csv')
resultado_experimento_2

,numero_inscricao,query
0,210061103945,"\nSELECT \n e.nu_inscricao,\n e.nu_nota_..."
1,210060214087,"\nSELECT \n b.no_municipio_esc,\n b.sg_u..."
2,210059980948,"\nSELECT \n no_municipio_esc,\n sg_uf_es..."
3,210060801601,"\nSELECT \n no_municipio_esc,\n sg_uf_es..."
4,210059085130,"\nSELECT \n no_municipio_esc,\n sg_uf_es..."
...,...,...
495,210058073319,"\nSELECT \n b.no_municipio_esc,\n b.sg_u..."
496,210059868183,"\nSELECT \n no_municipio_esc,\n sg_uf_es..."
497,210059085173,"\nSELECT \n no_municipio_esc,\n sg_uf_es..."
498,210059540806,"\nSELECT \n no_municipio_esc,\n sg_uf_es..."


In [ ]:
request(resultado_experimento_2, 'Experimento_2', 'query_sql_request.csv')

In [212]:
df = pd.read_csv('Resultados_Experimentos/Experimento_2/query_sql_request.csv')
df


,numero_inscricao,query,requisicao
0,210061103945,"\nSELECT \n e.nu_inscricao,\n e.nu_nota_...",Falha
1,210060214087,"\nSELECT \n b.no_municipio_esc,\n b.sg_u...",Sucesso
2,210059980948,"\nSELECT \n no_municipio_esc,\n sg_uf_es...",Sucesso
3,210060801601,"\nSELECT \n no_municipio_esc,\n sg_uf_es...",Falha
4,210059085130,"\nSELECT \n no_municipio_esc,\n sg_uf_es...",Sucesso
...,...,...,...
495,210058073319,"\nSELECT \n b.no_municipio_esc,\n b.sg_u...",Sucesso
496,210059868183,"\nSELECT \n no_municipio_esc,\n sg_uf_es...",Falha
497,210059085173,"\nSELECT \n no_municipio_esc,\n sg_uf_es...",Falha
498,210059540806,"\nSELECT \n no_municipio_esc,\n sg_uf_es...",Sucesso


In [ ]:
dic = {}

tam = int( (df.shape[0] * 0.05) + 1 )

for i in range(0, tam, 1):
    print(i)
    if df.loc[i, 'requisicao'] == 'Sucesso':

        numero_inscricao = df.loc[i, 'numero_inscricao']
        df_query = conn.sql(df.loc[i, 'query'])

        user_message_with_data = user_message_question.format(numero_inscricao) + 'Os dados com detalhes do desempenho do Aluno encontra-se abaixo \n ' + f' {df_query.fetchnumpy()} Ajude o aluno para melhorar sua nota: '

        resposta = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=message_LLM_Teacher(user_message_with_data),
            max_tokens=1000,
            temperature=0.7,
        )

        mensagem_resp = resposta.choices[0].message.content

        print(mensagem_resp)

        dic[str(numero_inscricao)] = mensagem_resp


In [219]:
df_llm_teacher = save_results_from_llm_teacher(dic)
df_llm_teacher['numero_inscricao'] = df_llm_teacher['numero_inscricao'].astype('int')
resultado = pd.merge(left = df, right = df_llm_teacher, how = 'left', on = 'numero_inscricao')
resultado.to_csv('Resultados_Experimentos/Experimento_2/resultado_final_recomendacao.csv', index = False)

## Experimento 3 

In [10]:
alunos_exp_3 = pd.read_csv('Bases_Experimentos/Experimento_3.csv')
nu_inscricao_aluno_exp_3 = alunos_exp_3['nu_inscricao'].values
#nu_inscricao_aluno_exp_2 = nu_inscricao_aluno_exp_2[0:5]
nu_inscricao_aluno_exp_3.shape

(500,)

In [37]:
user_message_question = '''
Sou o aluno {} desejava saber meu desempenho na Redação do Enem e em todas as competências para redação ?
Também queria saber informações estatísticas comparando meu desempenho com alunos do mesmo Estado que moro e a Cidade.
Por último queria uma análise do meu desempenho voltada com outros alunos da mesma faixa de renda de minha familia  ?'''

In [14]:
alunos_exp_3

,nu_inscricao,nu_nota_comp1,nu_nota_comp2,nu_nota_comp3,nu_nota_comp4,nu_nota_comp5,nu_nota_redacao
0,210061103945,140.0,200.0,100.0,120.0,140.0,700.0
1,210060214087,140.0,200.0,160.0,180.0,200.0,880.0
2,210059980948,120.0,120.0,120.0,120.0,80.0,560.0
3,210060801601,120.0,120.0,120.0,120.0,120.0,600.0
4,210059085130,140.0,200.0,160.0,160.0,200.0,860.0
...,...,...,...,...,...,...,...
495,210058073319,100.0,40.0,40.0,100.0,40.0,320.0
496,210059868183,160.0,200.0,200.0,200.0,200.0,960.0
497,210059085173,160.0,180.0,120.0,160.0,100.0,720.0
498,210059540806,140.0,180.0,140.0,140.0,120.0,720.0


In [240]:
dic = {}

i = 0

for numero_inscricao in nu_inscricao_aluno_exp_2:

    message = user_message_question.format(numero_inscricao)

    print(message)

    resposta = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=message_LLM_Sql(message),
        max_tokens=1000,
        temperature=0.7,
    )

    mensagem_resp = resposta.choices[0].message.content
    
    if i < 5: 
        print(mensagem_resp)

    sql_query = mensagem_resp.replace("```", "").split('sql')[1]

    dic[str(numero_inscricao)] = sql_query

    i = i + 1


Sou o aluno 210061103945 desejava saber meu desempenho na Redação do Enem e em todas as competências para redação ?
Também queria saber informações estatísticas comparando meu desempenho com alunos do mesmo Estado que moro e a Cidade.
Por último queria uma análise do meu desempenho voltada com outros alunos da mesma faixa de renda de minha familia  ?
```sql
SELECT 
    nu_inscricao,
    nu_nota_comp1,
    nu_nota_comp2,
    nu_nota_comp3,
    nu_nota_comp4,
    nu_nota_comp5,
    nu_nota_redacao,
    no_municipio_esc,
    sg_uf_esc,
    Q006
FROM 
    db.enem.base_enem
WHERE 
    nu_inscricao = 210061103945;

-- Estatísticas comparativas com alunos do mesmo estado e município
SELECT 
    AVG(nu_nota_redacao) AS media_nota_redacao,
    AVG(nu_nota_comp1) AS media_comp1,
    AVG(nu_nota_comp2) AS media_comp2,
    AVG(nu_nota_comp3) AS media_comp3,
    AVG(nu_nota_comp4) AS media_comp4,
    AVG(nu_nota_comp5) AS media_comp5
FROM 
    db.enem.base_enem
WHERE 
    sg_uf_esc = (SELECT sg_uf

In [15]:
resultado_experimento_3

NameError: name 'resultado_experimento_3' is not defined

In [17]:
#Salvar resultados
#save_results_from_llm_sql(dic , 'Experimento_3', 'query_sql.csv')

#Ler resultados
resultado_experimento_3 = pd.read_csv('Resultados_Experimentos/Experimento_3/query_sql_consolidado.csv')
resultado_experimento_3

,numero_inscricao,query
0,210061103945,"\nSELECT \n nu_inscricao,\n nu_nota_comp..."
1,210060214087,\nWITH aluno_desempenho AS (\n SELECT \n ...
2,210059980948,"\nSELECT \n a.nu_inscricao,\n a.nu_nota_..."
3,210060801601,"\nSELECT \n nu_inscricao,\n no_municipio..."
4,210059085130,\nWITH desempenho_aluno AS (\n SELECT \n ...
...,...,...
495,210058073319,"\nSELECT \n nu_inscricao,\n no_municipio..."
496,210059868183,"\nSELECT \n no_municipio_esc,\n sg_uf_es..."
497,210059085173,\nWITH aluno_dados AS (\n SELECT \n ...
498,210059540806,\nWITH desempenho_aluno AS (\n SELECT \n ...


In [ ]:
request(resultado_experimento_3, 'Experimento_3', 'query_sql_request.csv')

In [33]:
df = pd.read_csv('Resultados_Experimentos/Experimento_3/query_sql_request_consolidado.csv')
df

,numero_inscricao,query,requisicao
0,210061103945,"\nSELECT \n nu_inscricao,\n nu_nota_comp...",Sucesso
1,210060214087,\nWITH aluno_desempenho AS (\n SELECT \n ...,Sucesso
2,210059980948,"\nSELECT \n a.nu_inscricao,\n a.nu_nota_...",Sucesso
3,210060801601,"\nSELECT \n nu_inscricao,\n no_municipio...",Falha
4,210059085130,\nWITH desempenho_aluno AS (\n SELECT \n ...,Sucesso
...,...,...,...
495,210058073319,"\nSELECT \n nu_inscricao,\n no_municipio...",Sucesso
496,210059868183,"\nSELECT \n no_municipio_esc,\n sg_uf_es...",Sucesso
497,210059085173,\nWITH aluno_dados AS (\n SELECT \n ...,Sucesso
498,210059540806,\nWITH desempenho_aluno AS (\n SELECT \n ...,Sucesso


In [38]:
user_message_question

'\nSou o aluno {} desejava saber meu desempenho na Redação do Enem e em todas as competências para redação ?\nTambém queria saber informações estatísticas comparando meu desempenho com alunos do mesmo Estado que moro e a Cidade.\nPor último queria uma análise do meu desempenho voltada com outros alunos da mesma faixa de renda de minha familia  ?'

In [39]:
dic = {}

tam = int( (df.shape[0] * 0.05) + 1 )

for i in range(0, tam, 1):
    print(i)
    if df.loc[i, 'requisicao'] == 'Sucesso':

        numero_inscricao = df.loc[i, 'numero_inscricao']
        df_query = conn.sql(df.loc[i, 'query'])

        user_message_with_data = user_message_question.format(numero_inscricao) + 'Os dados com detalhes do desempenho do Aluno encontra-se abaixo \n ' + f' {df_query.fetchnumpy()} Ajude o aluno para melhorar sua nota: '

        resposta = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=message_LLM_Teacher(user_message_with_data),
            max_tokens=1000,
            temperature=0.7,
        )

        mensagem_resp = resposta.choices[0].message.content

        print(mensagem_resp)

        dic[str(numero_inscricao)] = mensagem_resp

0


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Vamos analisar seu desempenho na redação do Enem e nas diferentes competências, além de comparar seus resultados com os de outros alunos da mesma faixa de renda e da sua região.

### Desempenho na Redação
1. **Nota Total da Redação:** 592 pontos (soma das notas das competências)
2. **Nota das Competências:**
   - **Competência 1:** 118 (Domínio da modalidade escrita formal da Língua Portuguesa)
   - **Competência 2:** 134 (Compreensão da proposta de redação)
   - **Competência 3:** 113 (Seleção e organização de informações)
   - **Competência 4:** 124 (Uso de mecanismos linguísticos)
   - **Competência 5:** 101 (Elaboração de proposta de intervenção)

### Análise das Competências
#### Competência 1
- **Desempenho:** 118 pontos
- **Análise:** Você demonstra um domínio mediano da escrita formal, mas com alguns desvios gramaticais.
- **Sugestões de Melhoria:** Pratique a leitura e a escrita de textos formais. Revise regras de gramática e faça exercícios de revisão de textos.

#### Competê

...

### Desempenho na Redação do Enem

- **Nota Final da Redação**: 940 pontos
- **Nota por Competência**:
  - **Competência 1**: 160 pontos
  - **Competência 2**: 200 pontos
  - **Competência 3**: 180 pontos
  - **Competência 4**: 200 pontos
  - **Competência 5**: 200 pontos

### Análise das Competências

1. **Competência 1** (Domínio da modalidade escrita formal da Língua Portuguesa):
   - **Desempenho**: 160 pontos (Bom domínio)
   - **Sugestões de Melhoria**: Para avançar, é importante revisar regras gramaticais e de ortografia. Praticar a leitura e a escrita de textos dissertativos pode ajudar a identificar e corrigir erros comuns.

2. **Competência 2** (Compreensão da proposta de redação):
   - **Desempenho**: 200 pontos (Excelente domínio)
   - **Sugestões de Melhoria**: Continue a desenvolver argumentos consistentes e a utilizar repertório sociocultural variado. Você já está no caminho certo, então mantenha essa prática.

3. **Competência 3** (Seleção e organização de informações):

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

**Desempenho na Redação do Enem:**

Para o aluno com inscrição 210059085135, o desempenho na redação do Enem é o seguinte:

- **Nota Final da Redação**: 635,84
- **Competência 1**: 124,85 (Domínio da modalidade escrita formal da Língua Portuguesa)
- **Competência 2**: 143,37 (Compreensão da proposta de redação e desenvolvimento do tema)
- **Competência 3**: 121,90 (Seleção, organização e interpretação de informações)
- **Competência 4**: 132,69 (Conhecimento dos mecanismos linguísticos)
- **Competência 5**: 113,03 (Elaboração da proposta de intervenção)

**Análise das Competências:**

1. **Competência 1 (Domínio da Língua Portuguesa)**: Nota 124,85
   - O aluno apresenta um domínio mediano da língua, com alguns desvios gramaticais. Para melhorar, recomenda-se:
     - Estudar regras gramaticais específicas que apresentem dificuldades.
     - Praticar a escrita revisando textos e buscando feedback de professores ou colegas.

2. **Competência 2 (Desenvolvimento do Tema)**: Nota 143,37
   

In [40]:
df_llm_teacher = save_results_from_llm_teacher(dic)
df_llm_teacher['numero_inscricao'] = df_llm_teacher['numero_inscricao'].astype('int')
resultado = pd.merge(left = df, right = df_llm_teacher, how = 'left', on = 'numero_inscricao')
resultado.to_csv('Resultados_Experimentos/Experimento_3/resultado_final_recomendacao.csv', index = False)

Vamos analisar seu desempenho na redação do Enem e nas diferentes competências, além de comparar seus resultados com os de outros alunos da mesma faixa de renda e da sua região.

### Desempenho na Redação
1. **Nota Total da Redação:** 592 pontos (soma das notas das competências)
2. **Nota das Competências:**
   - **Competência 1:** 118 (Domínio da modalidade escrita formal da Língua Portuguesa)
   - **Competência 2:** 134 (Compreensão da proposta de redação)
   - **Competência 3:** 113 (Seleção e organização de informações)
   - **Competência 4:** 124 (Uso de mecanismos linguísticos)
   - **Competência 5:** 101 (Elaboração de proposta de intervenção)

### Análise das Competências
#### Competência 1
- **Desempenho:** 118 pontos
- **Análise:** Você demonstra um domínio mediano da escrita formal, mas com alguns desvios gramaticais.
- **Sugestões de Melhoria:** Pratique a leitura e a escrita de textos formais. Revise regras de gramática e faça exercícios de revisão de textos.

#### Competência 2
- **Desempenho:** 134 pontos
- **Análise:** Bom domínio, mas a argumentação pode ser aprimorada.
- **Sugestões de Melhoria:** Estude estruturas de textos dissertativos e como construir uma argumentação consistente. Utilize repertório sociocultural em suas redações.

#### Competência 3
- **Desempenho:** 113 pontos
- **Análise:** Apresenta informações limitadas e pouco organizadas.
- **Sugestões de Melhoria:** Pratique a construção de argumentos a partir de diferentes fontes e organize suas ideias de forma clara, seguindo uma linha de raciocínio lógica.

#### Competência 4
- **Desempenho:** 124 pontos
- **Análise:** Há articulação das partes do texto, porém algumas inadequações.
- **Sugestões de Melhoria:** Estude conectivos e a estrutura de parágrafos. Treine a coesão e a coerência nos seus textos.

#### Competência 5
- **Desempenho:** 101 pontos
- **Análise:** Proposta de intervenção insuficiente.
- **Sugestões de Melhoria:** Elabore propostas mais claras e viáveis, ligadas à discussão do texto. Utilize exemplos práticos e específicos.

### Comparação com Alunos da Sua Região e Renda
- **Média de Nota na Redação para a Faixa de Renda:** 592.04 pontos
- **Médias das Competências na Faixa de Renda:**
  - **Competência 1:** 118.23
  - **Competência 2:** 134.13
  - **Competência 3:** 113.68
  - **Competência 4:** 124.76
  - **Competência 5:** 101.24

### Análise Comparativa
- Seu desempenho em cada competência está próximo das médias dos estudantes da mesma faixa de renda, o que indica que você está dentro do padrão. No entanto, focar nas sugestões acima pode ajudar a superar as médias e aumentar suas notas.

### Recomendações Gerais
1. **Estudo Contínuo:** Dedique tempo para ler diferentes tipos de textos argumentativos e dissertativos.
2. **Prática de Redação:** Escreva redações regularmente, seguindo os temas propostos em provas anteriores do Enem.
3. **Feedback:** Peça a professores ou colegas que revisem suas redações e ofereçam feedback construtivo.
4. **Simulados:** Realize simulados para se familiarizar com o tempo e a estrutura da prova.

Ao seguir essas orientações e se dedicar ao estudo, você pode melhorar significativamente sua nota na redação do Enem. Boa sorte!